In [32]:
import functools, inspect

class object_proxy(object): 

    def __init__(self, wrapped):
        self.wrapped = wrapped
        try:
            self.__name__= wrapped.__name__
        except AttributeError:
            pass 

    @property
    def __class__(self):
        return self.wrapped.__class__  

    def __getattr__(self, name):
        return getattr(self.wrapped, name)
    
    
class bound_function_wrapper(object_proxy): 

    def __init__(self, wrapped, instance, wrapper, binding, parent):
        super(bound_function_wrapper, self).__init__(wrapped)
        self.instance = instance
        self.wrapper  = wrapper
        self.binding  = binding
        self.parent   = parent 

    def __call__(self, *args, **kwargs):
        if self.binding == 'function':
            if self.instance is None:
                instance, args = args[0], args[1:]
                wrapped = functools.partial(self.wrapped, instance)
                return self.wrapper(wrapped, instance, args, kwargs)
            else:
                return self.wrapper(self.wrapped, self.instance, args, kwargs)
        else:
            instance = getattr(self.wrapped, '__self__', None)
            return self.wrapper(self.wrapped, instance, args, kwargs) 

    def __get__(self, instance, owner):
        if self.instance is None and self.binding == 'function':
            descriptor = self.parent.wrapped.__get__(instance, owner)
            return bound_function_wrapper(descriptor, instance, self.wrapper,
                    self.binding, self.parent)
        return self 

    
class function_wrapper(object_proxy): 

    def __init__(self, wrapped, wrapper):
        super(function_wrapper, self).__init__(wrapped)
        self.wrapper = wrapper
        if isinstance(wrapped, classmethod):
            self.binding = 'classmethod'
        elif isinstance(wrapped, staticmethod):
            self.binding = 'staticmethod'
        else:
            self.binding = 'function' 

    def __get__(self, instance, owner):
        wrapped = self.wrapped.__get__(instance, owner)
        return bound_function_wrapper(wrapped, instance, self.wrapper, self.binding, self) 

    def __call__(self, *args, **kwargs):
        return self.wrapper(self.wrapped, None, args, kwargs)
    
    
def decorator(wrapper):
    @functools.wraps(wrapper)
    def _decorator(wrapped):
        return function_wrapper(wrapped, wrapper)
    return _decorator

@decorator
def universal(wrapped, instance, args, kwargs):
    if instance is None:
        if inspect.isclass(wrapped):
            # Decorator was applied to a class.
            return wrapped(*args, **kwargs)
        else:
            # Decorator was applied to a function or staticmethod.
            return wrapped(*args, **kwargs)
    else:
        if inspect.isclass(instance):
            # Decorator was applied to a classmethod.
            return wrapped(*args, **kwargs)
        else:
            # Decorator was applied to an instancemethod.
            return wrapped(*args, **kwargs)

In [28]:
@decorator
def my_function_wrapper(wrapped, instance, args, kwargs):
    print('WRAPPED',  wrapped)
    print('INSTANCE', instance)
    print('ARGS',     args)
    return wrapped(*args, **kwargs) 

@my_function_wrapper
def function(a, b):
    pass 

class Class(object):
    @my_function_wrapper
    def function_im(self, a, b):
        pass 

In [29]:
function(1, 2)

WRAPPED <function function at 0x10326a488>
INSTANCE None
ARGS (1, 2)


In [30]:
c = Class()
c.function_im(1, 2)

WRAPPED <bound method Class.function_im of <__main__.Class object at 0x1032589b0>>
INSTANCE <__main__.Class object at 0x1032589b0>
ARGS (1, 2)


In [31]:
Class.function_im(c, 1, 2)

WRAPPED functools.partial(<function Class.function_im at 0x10326aea0>, <__main__.Class object at 0x1032589b0>)
INSTANCE <__main__.Class object at 0x1032589b0>
ARGS (1, 2)


In [38]:
@my_function_wrapper
class Class(object):

    @my_function_wrapper
    def function_im(self, a, b):
        pass

    @my_function_wrapper
    @classmethod
    def function_cm(self, a, b):
        pass

    @my_function_wrapper
    @staticmethod
    def function_sm(a, b):
        pass 

c = Class()

WRAPPED <class '__main__.Class'>
INSTANCE None
ARGS ()


In [39]:
c.function_im(1,2)

WRAPPED <bound method Class.function_im of <__main__.Class object at 0x1032a0748>>
INSTANCE <__main__.Class object at 0x1032a0748>
ARGS (1, 2)


In [40]:
Class.function_im(c, 1, 2)

WRAPPED functools.partial(<function Class.function_im at 0x103147510>, <__main__.Class object at 0x1032a0748>)
INSTANCE <__main__.Class object at 0x1032a0748>
ARGS (1, 2)


In [41]:
c.function_cm(1,2)

WRAPPED <bound method Class.function_cm of <class '__main__.Class'>>
INSTANCE <class '__main__.Class'>
ARGS (1, 2)


In [42]:
Class.function_cm(1, 2)

WRAPPED <bound method Class.function_cm of <class '__main__.Class'>>
INSTANCE <class '__main__.Class'>
ARGS (1, 2)


In [43]:
c.function_sm(1,2)

WRAPPED <function Class.function_sm at 0x1031478c8>
INSTANCE None
ARGS (1, 2)


In [44]:
Class.function_sm(1, 2)

WRAPPED <function Class.function_sm at 0x1031478c8>
INSTANCE None
ARGS (1, 2)


In [45]:
def optional_arguments(wrapped=None, arg=1):
    if wrapped is None:
        return functools.partial(optional_arguments, arg=arg) 

    @decorator
    def _wrapper(wrapped, instance, args, kwargs):
        return wrapped(*args, **kwargs)

    return _wrapper(wrapped) 

@optional_arguments(arg=2)
def function1():
    pass

@optional_arguments
def function2():
    pass 

In [47]:
function2()